### Importing Files and Loading CIFAR10 Dataset

In [2]:
# importing packages
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import cv2 


In [3]:
cifar10 = keras.datasets.cifar10

(x_train, y_train) , (x_test, y_test) = cifar10.load_data()



### Creating CNN from Keras Sequential Models, Traing and Testing the model

In [41]:
cnn = tf.keras.models.Sequential([
  keras.layers.BatchNormalization(input_shape=(32,32,3)),
  keras.layers.Conv2D(filters= 32,kernel_size = (3,3) ,activation='relu'),
  keras.layers.MaxPool2D(),
  keras.layers.Conv2D(filters= 64,kernel_size = (3,3) ,activation='relu'),
  keras.layers.MaxPool2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(64),
  keras.layers.BatchNormalization(),
  keras.layers.ReLU(),
  keras.layers.Dense(10, activation='softmax')
])

In [10]:
cnn.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])


In [11]:
cnn.fit(x_train, y_train,epochs=5)

Epoch 1/5
1563/1563 [==============================] - 52s 32ms/step - loss: 1.2529 - accuracy: 0.5590
Epoch 2/5
1563/1563 [==============================] - 48s 31ms/step - loss: 0.9266 - accuracy: 0.6771
Epoch 3/5
1563/1563 [==============================] - 47s 30ms/step - loss: 0.7864 - accuracy: 0.7261
Epoch 4/5
1563/1563 [==============================] - 47s 30ms/step - loss: 0.6901 - accuracy: 0.7584
Epoch 5/5
1563/1563 [==============================] - 48s 30ms/step - loss: 0.6130 - accuracy: 0.7859


In [12]:
cnn.evaluate(x_test,  y_test, verbose=2)


313/313 - 3s - loss: 0.8630 - accuracy: 0.7089 - 3s/epoch - 8ms/step


[0.8629895448684692, 0.708899974822998]

### Loading VGG16 Pre-trained model and using transfer learnig to use this model on CIFAR10 dataset

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16


baseModelVGG16 = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
for layer in baseModelVGG16.layers:
     layer.trainable = False

x = keras.layers.GlobalAveragePooling2D()(baseModelVGG16.get_layer('block3_pool').output)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.6)(x)
pred = keras.layers.Dense(10, activation='softmax')(x)
VGG16model = keras.Model(baseModelVGG16.input, pred)



In [12]:
VGG16model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
VGG16model.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [7]:
x_trainResized = []
x_testResized = []
for img in x_test:
    imgResized = cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA)
    x_testResized.append(imgResized)

x_testResized =np.array(x_testResized)

for img in x_train:
    imgResized = cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA)
    x_trainResized.append(imgResized)


x_trainResized =np.array(x_trainResized)

x_testResized = x_testResized/225.0
x_trainResized = x_trainResized/225.0

In [13]:
VGG16model.fit(x_trainResized, y_train,epochs=5)

Epoch 1/5


2022-12-28 14:49:29.707733: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 738s 471ms/step - loss: 1.1484 - accuracy: 0.6017
Epoch 2/5
1563/1563 [==============================] - 742s 475ms/step - loss: 0.9366 - accuracy: 0.6779
Epoch 3/5
1563/1563 [==============================] - 823s 527ms/step - loss: 0.8690 - accuracy: 0.6990
Epoch 4/5
1563/1563 [==============================] - 850s 543ms/step - loss: 0.8249 - accuracy: 0.7159
Epoch 5/5
1563/1563 [==============================] - 875s 560ms/step - loss: 0.7858 - accuracy: 0.7284


In [14]:
VGG16model.evaluate(x_testResized,  y_test, verbose=2)

313/313 - 148s - loss: 0.6795 - accuracy: 0.7596 - 148s/epoch - 474ms/step


[0.6794503927230835, 0.7595999836921692]